.. _pytest_notebook_by_example:

# pytest-notebook by example

.. note::

    This tutorial is a notebook, that can be downloaded to run locally :download:`tutorial_intro.ipynb`,
    and was converted to documentation with [ipypublish](https://ipypublish.readthedocs.io)

## Python API

The principal component of `pytest-notebook` is the
:py:class:`~pytest_notebook.nb_regression.NBRegressionFixture` class,
which is an [attrs](http://www.attrs.org) class, whose parameters can be instatiated or set via attibutes.

In [1]:
import importlib_resources
from pytest_notebook import example_nbs
from pytest_notebook.nb_regression import NBRegressionFixture

In [2]:
fixture = NBRegressionFixture(exec_timeout=50)
fixture.diff_color_words = False
fixture

NBRegressionFixture(exec_notebook=True, exec_cwd=None, exec_allow_errors=False, exec_timeout=50, coverage=False, cov_config=None, cov_source=None, cov_merge=None, post_processors=('coalesce_streams',), process_resources={}, diff_replace=(), diff_ignore=('/cells/*/outputs/*/traceback',), diff_use_color=True, diff_color_words=False, force_regen=False)

The main method is :py:meth:`~pytest_notebook.nb_regression.NBRegressionFixture.check`, which executes a notebook and compares its initial and final contents.

In [3]:
with importlib_resources.path(example_nbs, "example1.ipynb") as path:
    fixture.check(str(path))

NBRegressionError: 
--- expected
+++ obtained
[34m## replaced /cells/1/execution_count:[0m
[31m-  2
[32m+  1

[0m[34m## modified /cells/2/outputs/0/text:[0m
[36m@@ -1,2 +1,2 @@[m
 hallo1[m
[31m-hallo3[m
[32m+[m[32mhallo2[m

[0m

To return the results, without raising an exception, use ``raise_errors=False``. This returns a :py:class:`~pytest_notebook.nb_regression.NBRegressionResult` instance.

In [4]:
with importlib_resources.path(example_nbs, "example1.ipynb") as path:
    result = fixture.check(str(path), raise_errors=False)

result

NBRegressionResult(diff_full_length=1,diff_filtered_length=1)

In [5]:
print(result.diff_string)


--- expected
+++ obtained
## replaced /cells/1/execution_count:
-  2
+  1

## modified /cells/2/outputs/0/text:
@@ -1,2 +1,2 @@
 hallo1
-hallo3
+hallo2




The diff of the notebooks is returned as a list of [nbdime](https://nbdime.readthedocs.io) `DiffEntry` objects.

In [6]:
result.diff_filtered

[{'op': 'patch',
  'key': 'cells',
  'diff': [{'op': 'patch',
    'key': 1,
    'diff': [{'op': 'replace', 'key': 'execution_count', 'value': 1}]},
   {'op': 'patch',
    'key': 2,
    'diff': [{'op': 'patch',
      'key': 'outputs',
      'diff': [{'op': 'patch',
        'key': 0,
        'diff': [{'op': 'patch',
          'key': 'text',
          'diff': [{'op': 'patch',
            'key': 1,
            'diff': [{'op': 'addrange', 'key': 5, 'valuelist': '2'},
             {'op': 'removerange', 'key': 5, 'length': 1}]}]}]}]}]}]}]

The notebooks are returned as :py:class:`nbformat.NotebookNode` instances.

In [7]:
result.nb_final

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': '# Markdown Cell'},
  {'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [],
   'source': 'from IPython import display'},
  {'cell_type': 'code',
   'execution_count': 2,
   'metadata': {},
   'outputs': [{'output_type': 'stream',
     'name': 'stdout',
     'text': 'hallo1\nhallo2\n'}],
   'source': "print('hallo1')\nprint('hallo2')"}],
 'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'language_info': {'name': 'python',
   'version': '3.6.7',
   'mimetype': 'text/x-python',
   'codemirror_mode': {'name': 'ipython', 'version': 3},
   'pygments_lexer': 'ipython3',
   'nbconvert_exporter': 'python',
   'file_extension': '.py'}},
 'nbformat': 4,
 'nbformat_minor': 2}

## pytest fixture

.. seealso::

    :py:mod:`pytest_notebook.ipy_magic`,
    for the notebook magic used to run pytest in a Jupyter Notebook.

In [8]:
%load_ext pytest_notebook.ipy_magic

A :py:class:`~pytest_notebook.nb_regression.NBRegressionFixture` instance can accessed *via* the :py:func:`~pytest_notebook.plugin.nb_regression` fixture.
This instance will be instatiated with parameters dictated by arguments parsed from the pytest command-line and configuration file(s).

.. note::

    pytest-notebook command-line and configuration file parameter names
    are the same as for ``NBRegressionFixture``, but with the prefix ``nb_``.
    
    The command-line parameter takes precedence over the configuration file one.

In [9]:
%%pytest -v --color=yes --disable-warnings --nb-exec-timeout 50

---
[pytest]
nb_diff_color_words = True
---

import importlib_resources
from pytest_notebook import example_nbs

def test_notebook(nb_regression):
    with importlib_resources.path(example_nbs, "example1.ipynb") as path:
        nb_regression.check(str(path))

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmppyh8kyts, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 1 item

test_ipycell.py::test_notebook FAILED                                    [100%]

=================================== FAILURES ===================================
________________________________ test_notebook _________________________________

nb_regression = NBRegressionFixture(exec_notebook=True, exec_cwd='/Users/cjs14/GitHub/pytest-notebook/pytest_notebook/example_nbs', ex...place=(), diff_ignore=('/cells/*/outputs/*/traceback',), diff_use_color=True, diff_color_words=True, force_regen=False)

    def test_notebook(nb_regression):
        with importlib_resources.path(examp

## pytest file collection

:py:meth:`~pytest_notebook.nb_regression.NBRegressionFixture.check` can be run automatically on all notebooks using the pytest collection mechanism.
To activate this feature, set `--nb-test-files` on the command-line, or `nb_test_files = True` in the configuration file.

In [10]:
notebook1_content = importlib_resources.read_text(example_nbs, "example1_pass.ipynb")
notebook2_content = importlib_resources.read_text(example_nbs, "example1.ipynb")

In [11]:
%%pytest -v --color=yes --disable-warnings --nb-test-files

***
(notebook1_content, "test_notebook1.ipynb")
(notebook2_content, "test_notebook2.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpvs4w3kxu
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 2 items

test_notebook1.ipynb::nbregression(test_notebook1) PASSED                [ 50%]
test_notebook2.ipynb::nbregression(test_notebook2) FAILED                [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook2) ____________________
pytest_notebook.nb_regression.NBRegressionError: 
--- expected
+++ obtained
## replaced /cells/1/execution_count:
-  2
+  1

## modified /cells/2/outputs/0/text:
@@ -1,2 +1,2 @@
 hallo1
-hallo3
+hallo2


================ 1 failed, 1 passed, 1 warnings in 3.64 seconds ============

In [12]:
%%pytest -v --color=yes --disable-warnings

---
[pytest]
nb_test_files = True
---

***
(notebook1_content, "test_notebook1.ipynb")
(notebook2_content, "test_notebook2.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmphhv85_77, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 2 items

test_notebook1.ipynb::nbregression(test_notebook1) PASSED                [ 50%]
test_notebook2.ipynb::nbregression(test_notebook2) FAILED                [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook2) ____________________
pytest_notebook.nb_regression.NBRegressionError: 
--- expected
+++ obtained
## replaced /cells/1/execution_count:
-  2
+  1

## modified /cells/2/outputs/0/text:
@@ -1,2 +1,2 @@
 hallo1
-hallo3
+hallo2


================ 1 failed, 1 passed, 1 warnings in 3.76

To restrict the notebook files pytest collects, one or more filename pattern matches can also be set (see :py:mod:`fnmatch`).

In [13]:
%%pytest -v --color=yes --disable-warnings

---
[pytest]
nb_test_files = True
nb_file_fnmatch = test_*.ipynb tutorial_*.ipynb
---

***
(notebook1_content, "test_notebook1.ipynb")
(notebook2_content, "other_notebook2.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpd18whmsj, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 1 item

test_notebook1.ipynb::nbregression(test_notebook1) PASSED                [100%]

===================== 1 passed, 1 warnings in 2.31 seconds =====================


## Live Logging of Cell Execution

If you wish to view the progress of the notebook execution, you can use the [pytest live-logging](https://docs.pytest.org/en/latest/logging.html#live-logs) functionality:

In [14]:
%%pytest -v --color=yes --disable-warnings --nb-test-files --log-cli-level=info

***
(notebook1_content, "test_notebook1.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpzax121qh
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 1 item

test_notebook1.ipynb::nbregression(test_notebook1) 
-------------------------------- live log call ---------------------------------
INFO     pytest_notebook.execution:execution.py:142 About to execute notebook with 3 cells
INFO     pytest_notebook.execution:execution.py:148 Executing notebook with kernel: python3
INFO     pytest_notebook.execution:execution.py:168 Executing cell 1
INFO     pytest_notebook.execution:execution.py:168 Executing cell 2
PASSED                                                                   [100%]

===================== 1 passed, 1 warnings in 2.40 seconds ==========

.. _regen_notebooks:

## Regenerating Notebooks

Failing notebooks can be regenerated by setting `--nb-force-regen`.
This will overwrite failing notebooks with the output from the notebook execution.

.. note::

    Notebooks will not be regenerated if they raise any unexpected exceptions,
    during execution.
    
    This approach to regeneration mimics [pytest-regressions](https://pytest-regressions.readthedocs.io).

In [15]:
%%pytest -v --color=yes --disable-warnings --nb-force-regen

---
[pytest]
nb_test_files = True
nb_force_regen = True
---

***
(notebook1_content, "test_notebook1.ipynb")
(notebook2_content, "test_notebook2.ipynb")
***

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
NB force regen: True
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmp7n8y_fg0, inifile: pytest.ini
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 2 items

test_notebook1.ipynb::nbregression(test_notebook1) PASSED                [ 50%]
test_notebook2.ipynb::nbregression(test_notebook2) FAILED                [100%]

=================================== FAILURES ===================================
____________________ notebook: nbregression(test_notebook2) ____________________
pytest_notebook.nb_regression.NBRegressionError: Files differ and --nb-force-regen set, regenerating file at:
- /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmp7n8y_fg0/test_notebook2.ipynb
----------------------------- Captur

The regenration can be observed, if we run two tests on the same notebook.

In [16]:
%%pytest -v --color=yes --disable-warnings

import os, tempfile
import importlib_resources
import pytest
from pytest_notebook import example_nbs

@pytest.fixture(scope="module")
def notebook():
    tmphandle, tmppath = tempfile.mkstemp(suffix=".ipynb")
    with open(tmppath, "w") as handle:
        handle.write(
            importlib_resources.read_text(example_nbs, "example1.ipynb")
        )
    yield tmppath
    os.remove(tmppath)
    

def test_notebook1(nb_regression, notebook):
    nb_regression.force_regen = True
    nb_regression.check(notebook)

def test_notebook2(nb_regression, notebook):
    nb_regression.check(notebook)

============================= test session starts ==============================
platform darwin -- Python 3.6.7, pytest-5.0.1, py-1.8.0, pluggy-0.12.0 -- //anaconda/envs/pytest_nb/bin/python
cachedir: .pytest_cache
rootdir: /private/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T/tmpeb7sjuqh
plugins: cov-2.7.1, datadir-1.3.0, regressions-1.0.5, notebook-0.5.1
collecting ... collected 2 items

test_ipycell.py::test_notebook1 FAILED                                   [ 50%]
test_ipycell.py::test_notebook2 PASSED                                   [100%]

=================================== FAILURES ===================================
________________________________ test_notebook1 ________________________________

nb_regression = NBRegressionFixture(exec_notebook=True, exec_cwd='/var/folders/dm/b2qnkb_n3r72slmpxlfmcjvm00lbnd/T', exec_allow_errors...place=(), diff_ignore=('/cells/*/outputs/*/traceback',), diff_use_color=True, diff_color_words=False, force_regen=True)
notebook = '/var/folde